<h1> Guided lab: Regression Models with Keras</h1>

Objective for this Notebook

- How to use the Tensorflow API, Keras, to build a regression model.
- Download and Clean dataset
- Build a Neural Network
- Train and Test the Network   

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Download and Clean Dataset</a>  
2. <a href="#item32">Import Keras</a>  
3. <a href="#item33">Build a Neural Network</a>  
4. <a href="#item34">Train and Test the Network</a>  

</font>
</div>


<a id="item31"></a>


## Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [33]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's download the data and read it into a <em>pandas</em> dataframe.


In [ ]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')

concrete_data.head()

So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


#### Let's check how many data points we have.


In [ ]:
concrete_data.shape

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [ ]:
concrete_data.dtypes

In [ ]:
concrete_data.isnull().sum()

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [38]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength

target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [ ]:
predictors.head()

In [ ]:
target.head()

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [ ]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()

predictors_norm.head()

Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [ ]:
n_cols = predictors_norm.shape[1] # number of predictors

n_cols

<a id="item1"></a>


<a id='item32'></a>


## Import Keras


Recall from the videos that Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, you will have to install TensorFlow first and when you import the Keras library, it will be explicitly displayed what backend was used to install the Keras library. In CC Labs, we used TensorFlow as the backend to install Keras, so it should clearly print that when we import Keras.


#### Let's go ahead and import the Keras library


In [43]:
from tensorflow import keras

# OLD OUTPUT:
# 2024-08-14 17:38:10.878028: I tensorflow/core/platform/cpu_feature_guard.cc:210]
# This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
# To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.

As you can see, the TensorFlow backend was used to install the Keras library.


Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.


In [44]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>


## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [45]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has two hidden layers, each of 50 hidden units.


<a id="item4"></a>


<a id='item34'></a>


## Train and Test the Network


Let's call the function now to create our model.


In [ ]:
# build the model
model = regression_model()

Now we will split our data into training and testing with a 70%-30% split (test_size=0.3).   

70% of our data will be used for training :

X_train : Our training data containing the variables the model will use to learn how to predict.

y_train : Our training data containing the labels.

#
30% of our data will be used for testing its performance metrics :

X_test : Unseen data by the model, we will use it to make predictions with our trained model.

y_test : The labels associated with X_test data, we will compare them with the predictions that our model provided as output when shown X_test and that way we will obtain performance metrics.

In [47]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size = 0.3, random_state = 42)

Next, we will train the model using the *fit* method. We will train the model for 100 epochs.


In [ ]:
# fit the model
model.fit(X_train, y_train, epochs = 100, verbose = 2)

### Now lets evaluate our model and get performance metrics !

First, since our model only saw the training data and learned from it, we will make predictions on the unseen test set.

y_pred are the predictions that our model generated from it's experience when shown the predictor variables of X_test.

That way we can compare y_pred (predictions made by our model) with y_test (the real labels of X_test) and see the difference as performance metrics.

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

Since this is a regression model we will be using mean squared error, mean absolute error and the r squared score to evaluate it.

In [ ]:
# Calculate and print regression metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'R² Score: {r2:.4f}')

Feel free to vary the following and note what impact each change has on the model's performance:

1. Increase or decreate number of neurons in hidden layers
2. Add more hidden layers
3. Increase number of epochs


For other models, other types of performance metrics are more meaningfull. It all depends on context and the end goal of the model. 

Research, planning and analysing various metrics with a clearly defined end goal is key to understanding how capable a model is and what are his limitations.

Keep that in mind when tackling the next exercises!

### Thank you for following this tutorial!